In [75]:
from __future__ import division
import os
import subprocess
import commands
import gzip
import glob
import math
import pandas as pd
import numpy as np
import matplotlib as plt
from pyveplot import *
#from collections import namedtuple
import networkx as nx
import random
from IPython.display import SVG
%matplotlib inline

## Circos vs Hiveplots

This notebook attemps to find alternative, clearer plots for inter and intra-chromosomal structural variations. In other words, the idea is to go from

A typical Circos plot:

<img src="https://www.genomatix.de/online_help/help_regionminer/SV_circos_genome.png" height=300 width=300/>

To this:

<img src="img/hyplot_intra_inter.png"/>

From this:

<img src="img/sv_table.png"/>

Some of the preliminary data comes from [Australian Pancreatic Cancer Genome Initiative](http://www.pancreaticcancer.net.au/), other from the [ICGC-TCGA DREAM challenge](https://www.synapse.org/#!Synapse:syn312572) processed via the [bcbio cancer pipeline](https://bcbio-nextgen.readthedocs.org/en/latest/contents/pipelines.html#cancer-variant-calling). That pipeline run takes a considerable amount of time to run given the big input sizes and [running several variant callers](http://bcb.io/2015/03/05/cancerval/).

For pedagogical reasons, the resulting tab-separated `.tsv` files have been generated for easy analysis. If a more upstream run or (re)-analysis  is required, there's [reduced dataset that focuses on chromosome 6](https://bcbio-nextgen.readthedocs.org/en/latest/contents/teaching.html).

After filtering VCF's resulting from both [Manta](https://github.com/Illumina/manta) variant caller and [Lumpy](https://github.com/arq5x/lumpy-sv) with a simple [vawk expression](https://github.com/cc2qe/vawk):

<pre>
gzcat manta-variants.vcf.gz | vawk '{{if (($12 == "PASS")) print $1, $4, $11, $17 }}'
</pre>

A svtools script, [`vcfToBedpe`](https://raw.githubusercontent.com/hall-lab/svtools/master/bin/vcftobedpe), was used to convert a plain [VCF](https://samtools.github.io/hts-specs/VCFv4.2.pdf) to [BEDPE format](http://bedtools.readthedocs.org/en/latest/content/general-usage.html#bedpe-format) to generate a paired end version where structural variations are seen as pairs (`chrom` and `chrom_b` columns).

Finally, [pyveplot](https://github.com/CSB-IG/pyveplot), a Python implementation of [Hive plots](http://www.hiveplot.net/) was used to show the representation above.

A Hive plot is a perceptually uniform and scalable linear layout visualization for network visual analytics (doi: 10.1093/bib/bbr069).

<img src="img/hiveplot-thisisuseful.png"/>

In [60]:
vcf_data = "data/vcf"
tsv_data = "data/tsv"
ev_data = "data/sv_size"

event_colors = {'DEL': 'red',
                'INV': 'yellow',
                'DUP': 'blue',
                'BND': 'green',
                'complex': 'purple'}

## Plot a hiveplot

In [3]:
def hiveplot(fname, dataframe):
    # Remove duplicates and filter out ALTS (GL000226.1, GL000224.1 ...)
    #dataframe = dataframe[~dataframe["chrom"].str.contains("GL")]
    dataframe = dataframe.drop_duplicates(keep="first") ## XXX: Perhaps should group/count dups better?
    
    # a network
    g = nx.Graph()

    # our hiveplot object
    h = Hiveplot('{}.svg'.format(fname))

                  # start      end
    axis0 = Axis((200,200), (200,100), stroke="grey")
    axis1 = Axis((200,200), (300,300), stroke="blue", stroke_width=1.2)
    axis2 = Axis((200,200), (10,310), stroke="black", stroke_width=3)

    h.axes = [ axis0, axis1, axis2 ]
    
    #print "Structural variation events for ''{fname}'' have the following counts:\n\n{groupby}\n".format(
    #       groupby=dataframe.groupby("sv").count(), fname=fname)
    
    for row in dataframe.itertuples():
        # idx, u'sample', u'chrom', u'chrom_b', u'sv', u'counts'
        g.add_node(row[2])
        # Count = 1 looks better than parametrized with groupby
        g.add_edge(row[2], row[3], event=row[4], count=1)

    for n in g.nodes():
        # Separate instances for the axis, otherwise arcs go to itself.
        node = Node(n)
        node2 = Node(n)
        node3 = Node(n)

        # XXX: Find a better (more uniform) function than ord? 
        # A small hash function would be prob better here.
        # Calculates the offset of the chromosomes in the axis.

        off = 120
        n = str(n)
        
        if len(n) == 1:
            offset_axis0 = ord(n)
            offset_axis1 = ord(n)
            offset_axis2 = ord(n)
        else:
            chrom_offset = 0
            for char in n:
                chrom_offset = chrom_offset + ord(char)

            offset_axis0 = chrom_offset
            offset_axis1 = chrom_offset
            offset_axis2 = chrom_offset

        offset_axis0 = offset_axis0/off
        offset_axis1 = offset_axis1/off
        offset_axis2 = offset_axis2/off

        axis0.add_node(node, offset_axis0)
        axis1.add_node(node2, offset_axis1)
        axis2.add_node(node3, offset_axis2)

    for e in g.edges():
        edge_data = g.get_edge_data(*e)

        # inter-chromosomal axis
        if e[0] != e[1] and (e[0] in axis0.nodes) and (e[1] in axis1.nodes):
            h.connect(axis0, e[0], 45, 
                      axis1, e[1], -45, 
                      stroke_width=edge_data['count'], 
                      stroke=event_colors[edge_data['event']])
        
        # intra-chromosomal axis
        elif e[0] == e[1] and (e[0] in axis1.nodes) and (e[1] in axis2.nodes):
            h.connect(axis1, e[0], 15, 
                      axis2, e[1], -15, 
                      stroke_width=edge_data['count'], 
                      stroke=event_colors[edge_data['event']])

    h.save()

### Shipped TSV's do not have the same structure, normalize

In [4]:
def normalize(data):    
    if "counts" not in data.columns:
        data.columns = ["chrom", "chrom_b", "sv"]
        data.insert(0, 'sample', np.nan)
        data.insert(len(data.columns), 'counts', np.nan)
    else:
        data.insert(2, 'chrom_b', 0)

    # Cleanup GL* alts
    data = data[~data["chrom"].str.contains("GL")]
    
    return data

In [61]:
%%time
for tumor in glob.iglob(os.path.join(vcf_data, "*_Tumor*.vcf.gz")):
    # already converted to bedpe
    if 'paired' in tumor:
        continue
    base_tumor_fn = os.path.splitext(os.path.splitext(tumor)[0]) 
    paired_fn = base_tumor_fn[0]+".paired"+".vcf"
    final_tsv = os.path.join(tsv_data, os.path.basename(base_tumor_fn[0])+".tsv")
    sv_size = os.path.join(ev_data, os.path.basename(base_tumor_fn[0])+".tsv")
    
    vcftope_cmd = (["vcftobedpe", "-i", tumor, "-o", paired_fn])
    
    vawk_cmd = """vawk '{{if (($12 == "PASS" || $7 == ".") && (S$GT != "0/0") && \
                  (S$SR > 5)) print $1,$4,$11,$17}}' {paired_vcf} | \
                  grep -v GL > {tumor_tsv}""".format(paired_vcf=paired_fn, tumor_tsv=final_tsv)
    
    vawk_sv_size = """vawk '{{if (($12 == "PASS" || $7 == ".") && (S$GT != "0/0") && \
                  (S$SR > 5)) print $1,$2,$4,$5,$11,$17}}' {paired_vcf} | \
                  grep -v GL > {tumor_tsv}""".format(paired_vcf=paired_fn, tumor_tsv=sv_size)
    
    subprocess.check_call(vcftope_cmd)
    commands.getstatusoutput(vawk_cmd)
    commands.getstatusoutput(vawk_sv_size)

CPU times: user 3.62 ms, sys: 20.7 ms, total: 24.3 ms
Wall time: 14.8 s


In [52]:
def sample_summary_table(datasets):
    """ A table with chr1-Y as rows, each sample as a column, 
        number of inter/intra-chromosomal events as data points.

        :datasets: a tuple with several (data, sample_names) where data is the associated Pandas dataframe.
    """
    chromosomes = [str(x) for x in range(1,23)]
    chromosomes.append('X')
    chromosomes.append('Y')
    
    samples = [x[1] for x in datasets]
    
    summary = pd.DataFrame(columns=samples, index=chromosomes)
    
    for data, sample in datasets:
        # Uncomment and tweak code below if one wants totals, not by chromosome
        #intra_chrom = len(data[data['chrom'] == data['chrom_b']].index)
        #inter_chrom = len(data[data['chrom'] != data['chrom_b']].index)
        
        for chrom in chromosomes:
            intra_specific_chrom = len(data[((data['chrom'] == chrom) | (data['chrom_b'] == chrom)) & (data['chrom'] == data['chrom_b'])].index)
            inter_specific_chrom = len(data[((data['chrom'] == chrom) | (data['chrom_b'] == chrom)) & (data['chrom'] != data['chrom_b'])].index)
            
            summary.loc[chrom][sample] = (intra_specific_chrom, inter_specific_chrom)
        
        #print data.groupby(['sv', 'chrom'])['chrom'].count()
        
    return summary

In [141]:
def event_size_distr(dataset):
    """Can you get the size distribution of events out of the VCF (for the intrachromosomal SVs only)? 
       Wonder if this is Lumpy calling tons of short to mid-sized Indels. 
    """
    df = pd.read_table(dataset, dtype=object, names = ["chrom_a", "pos_a", "chrom_b", "pos_b", "sv"])
    # Only intra-chromosomal, positions between distant chroms can be misleading
    df = df[df['chrom_a'] == df['chrom_b']]
    df["size"] = abs(np.int64(df["pos_b"]) - np.int64(df["pos_a"]))
    #size_distr = df.groupby(['sv', 'chrom_a'])['size']

    return df

### Process and plot all the TSV's

In [145]:
samples = []
samples_size_distr = []

for dataset in glob.iglob(os.path.join(tsv_data, "*.tsv")):
    dataset_name = os.path.basename(dataset)
    event_sizes = os.path.join(ev_data, os.path.basename(dataset))
    
    study = pd.read_table(dataset, dtype=object)
    study = normalize(study)
    
    samples.append((study, dataset_name))
    samples_size_distr.append((event_sizes, event_size_distr(event_sizes)))
    
    # plot them all
    hiveplot(dataset_name, study)

#sample_summary_table(samples)
for fname, data in samples_size_distr:
    print fname
    #print data[['chrom_a', 'chrom_b', 'sv', 'size']]
    print data.groupby(['sv'])['size'].describe()

data/sv_size/APGI1953_Tumor-lumpy.tsv
sv        
BND  count    2.380000e+02
     mean     1.715578e+07
     std      3.309279e+07
     min      5.000000e+01
     25%      1.712500e+02
     50%      2.058285e+05
     75%      1.675164e+07
     max      1.798211e+08
DEL  count    3.490000e+02
     mean     5.589061e+06
     std      1.956366e+07
     min      1.900000e+01
     25%      7.600000e+01
     50%      3.100000e+02
     75%      8.263000e+03
     max      1.117998e+08
DUP  count    1.770000e+02
     mean     1.170300e+07
     std      3.051701e+07
     min      1.070000e+02
     25%      5.390000e+02
     50%      3.233000e+03
     75%      8.744100e+04
     max      1.470550e+08
INV  count    3.000000e+01
     mean     7.778667e+02
     std      3.625828e+03
     min      3.400000e+01
     25%      7.025000e+01
     50%      8.450000e+01
     75%      1.130000e+02
     max      1.996900e+04
dtype: float64
data/sv_size/APGI1953_Tumor-manta.tsv
sv        
DEL  count          51.